In [30]:
import os
import json
import numpy as np
import pandas as pd
import scipy.io as sio

In [2]:
# Panasonic Data
panasonic = "..//data//Panasonic//Panasonic 18650PF Data"
all_temperatures = ["0degC", "-10degC", "-20degC", "10degC", "25degC"]

# dropped files are just concats of previous measurements
dropped = [
    '06-01-17_10.36 0degC_LA92_NN_Pan18650PF.mat',
    '03-27-17_09.06 10degC_US06_HWFET_UDDS_LA92_NN_Pan18650PF.mat',
    '06-07-17_08.39 n10degC_US06_HWFET_UDDS_LA92_Pan18650PF.mat',
    '06-23-17_23.35 n20degC_HWFET_UDDS_LA92_NN_Pan18650PF.mat',
]

In [3]:
def read_mat_file(file):
    df = pd.DataFrame()
    columns = ['TimeStamp', 'Voltage', 'Current', 'Ah', 'Wh', 'Power', 'Battery_Temp_degC', 'Time', 'Chamber_Temp_degC']
    for col in columns:
       df[col] = pd.Series(file['meas'][0][0][col].flatten())
    return df

# Include features

In [4]:
def symmetric_derivative(y, t):
    """
    Calculates the symmetric difference quotient.
    """
    y, t = np.array(y), np.array(t)
    
    delta_2t = np.diff(t)[1:] + np.diff(t)[:-1]
    forward_step = y[2:]
    backward_step = y[:-2]
    
    return  np.concatenate(([np.nan], (forward_step - backward_step)/delta_2t, [np.nan]))

In [16]:
# Concat all drive cycle data
df_drive_cycle_list = []
for temp in all_temperatures:
    panasonic_dir = panasonic + f"//{temp}" + "//Drive cycles"
    panasonic_files = [panasonic_dir + "//" + f for f in os.listdir(panasonic_dir) if f not in dropped]
    for file in panasonic_files:
        df_file = read_mat_file(sio.loadmat(file))
        
        # calculate SoC given a Panasonic 18650PF cell with a maximum capacity of 2.9Ah
        # Assumption: All test were started at an initial 100% SoC
        df_file["SoC"] = (2.9 + df_file['Ah'])/2.9
        
        # Moving Averages
        df_file["Voltage_MA1000"] = df_file["Voltage"].rolling(1000,min_periods=1).mean()
        df_file["Current_MA1000"] = df_file["Current"].rolling(1000,min_periods=1).mean()
        df_file["Voltage_MA400"] = df_file["Voltage"].rolling(400,min_periods=1).mean()
        df_file["Current_MA400"] = df_file["Current"].rolling(400,min_periods=1).mean()
        df_file["Voltage_MA200"] = df_file["Voltage"].rolling(200,min_periods=1).mean()
        df_file["Current_MA200"] = df_file["Current"].rolling(200,min_periods=1).mean()
        df_file["Voltage_MA100"] = df_file["Voltage"].rolling(100,min_periods=1).mean()
        df_file["Current_MA100"] = df_file["Current"].rolling(100,min_periods=1).mean()
        df_file["Voltage_MA50"] = df_file["Voltage"].rolling(50,min_periods=1).mean()
        df_file["Current_MA50"] = df_file["Current"].rolling(50,min_periods=1).mean()
        df_file["Voltage_MA10"] = df_file["Voltage"].rolling(10,min_periods=1).mean()
        df_file["Current_MA10"] = df_file["Current"].rolling(10,min_periods=1).mean()
                
        # Power
        df_file["Power"] = df_file["Voltage"]*df_file["Current"]
        
        # Derivatives
        df_file["Voltage_grad"] = symmetric_derivative(df_file["Voltage"], df_file["Time"])
        df_file["Current_grad"] = symmetric_derivative(df_file["Current"], df_file["Time"])
        df_file["Battery_Temp_grad"] = symmetric_derivative(df_file["Battery_Temp_degC"], df_file["Time"])
        df_file["Power_grad"] = symmetric_derivative(df_file["Power"], df_file["Time"])
        
        # Power
        df_file["Power"] = df_file["Voltage"]*df_file["Current"] 
        df_file = df_file.drop(['Chamber_Temp_degC'], axis=1)
        
        df_drive_cycle_list += [df_file.dropna()]      

df_drive_cycle = pd.concat(df_drive_cycle_list)

In [18]:
df_drive_cycle

,TimeStamp,Voltage,Current,Ah,Wh,Power,Battery_Temp_degC,Time,SoC,Voltage_MA1000,...,Voltage_MA100,Current_MA100,Voltage_MA50,Current_MA50,Voltage_MA10,Current_MA10,Voltage_grad,Current_grad,Battery_Temp_grad,Power_grad
1,[5/30/2017 12:56:00 PM],4.06607,-1.83413,-0.00005,-0.00022,-7.457701,0.335766,0.100995,0.999983,4.101135,...,4.101135,-1.815755,4.101135,-1.815755,4.101135,-1.815755,-0.586067,-0.335560,0.000000,-0.294836
2,[5/30/2017 12:56:00 PM],4.01782,-1.86516,-0.00010,-0.00043,-7.493877,0.335766,0.201990,0.999966,4.073363,...,4.073363,-1.832223,4.073363,-1.832223,4.073363,-1.832223,-0.405672,-0.269512,0.000000,-0.329606
3,[5/30/2017 12:56:00 PM],3.98372,-1.88884,-0.00016,-0.00064,-7.524610,0.335766,0.303992,0.999945,4.050953,...,4.050953,-1.846378,4.050953,-1.846378,4.050953,-1.846378,-0.297432,-0.205173,0.000000,-0.257443
4,[5/30/2017 12:56:00 PM],3.95863,-1.90599,-0.00021,-0.00085,-7.545109,0.335766,0.400994,0.999928,4.032488,...,4.032488,-1.858300,4.032488,-1.858300,4.032488,-1.858300,-0.217719,-0.156711,0.000000,-0.206302
5,[5/30/2017 12:56:00 PM],3.94061,-1.91987,-0.00026,-0.00105,-7.565459,0.335766,0.501999,0.999910,4.017175,...,4.017175,-1.868562,4.017175,-1.868562,4.017175,-1.868562,-0.152113,-0.112657,0.892205,-0.152560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116976,[3/21/2017 7:42:46 PM],3.35272,0.00000,-2.54962,-8.63844,0.000000,27.491260,11732.828998,0.120821,3.352025,...,3.352720,0.000000,3.352720,0.000000,3.352720,0.000000,0.003062,0.000000,-0.053636,0.000000
116977,[3/21/2017 7:42:46 PM],3.35336,0.00000,-2.54962,-8.63844,0.000000,27.480050,11732.930999,0.120821,3.352027,...,3.352726,0.000000,3.352733,0.000000,3.352784,0.000000,0.003184,0.000000,-0.055770,0.000000
116978,[3/21/2017 7:42:46 PM],3.35336,0.00000,-2.54962,-8.63844,0.000000,27.480050,11733.030003,0.120821,3.352030,...,3.352733,0.000000,3.352746,0.000000,3.352848,0.000000,0.000000,0.000000,0.000000,0.000000
116979,[3/21/2017 7:42:46 PM],3.35336,0.00000,-2.54962,-8.63844,0.000000,27.480050,11733.128996,0.120821,3.352033,...,3.352739,0.000000,3.352758,0.000000,3.352912,0.000000,-0.003232,0.000000,-1.075520,0.000000


In [19]:
df_drive_cycle.columns

Index(['TimeStamp', 'Voltage', 'Current', 'Ah', 'Wh', 'Power',
       'Battery_Temp_degC', 'Time', 'SoC', 'Voltage_MA1000', 'Current_MA1000',
       'Voltage_MA400', 'Current_MA400', 'Voltage_MA200', 'Current_MA200',
       'Voltage_MA100', 'Current_MA100', 'Voltage_MA50', 'Current_MA50',
       'Voltage_MA10', 'Current_MA10', 'Voltage_grad', 'Current_grad',
       'Battery_Temp_grad', 'Power_grad'],
      dtype='object')

# Normalization

In [21]:
features_to_norm = ['Voltage', 'Current', 'Power', 'Battery_Temp_degC', 'Voltage_MA1000', 'Current_MA1000', 'Voltage_MA400', 'Current_MA400', 'Voltage_MA200', 'Current_MA200', 'Voltage_MA100', 'Current_MA100', 'Voltage_MA50', 'Current_MA50', 'Voltage_MA10', 'Current_MA10', 'Voltage_grad', 'Current_grad', 'Battery_Temp_grad', 'Power_grad']

In [23]:
norm_basis = {}
for feature in features_to_norm:
    minmax = (min(df_drive_cycle[feature]), max(df_drive_cycle[feature]))
    norm_basis[feature] = minmax
    df_drive_cycle[feature] = (df_drive_cycle[feature] - minmax[0])/(minmax[1] - minmax[0])

In [27]:
df_drive_cycle.head(5)

,TimeStamp,Voltage,Current,Ah,Wh,Power,Battery_Temp_degC,Time,SoC,Voltage_MA1000,...,Voltage_MA100,Current_MA100,Voltage_MA50,Current_MA50,Voltage_MA10,Current_MA10,Voltage_grad,Current_grad,Battery_Temp_grad,Power_grad
1,[5/30/2017 12:56:00 PM],0.873893,0.605259,-0.00005,-0.00022,0.510177,0.389955,0.100995,0.999983,0.948782,...,0.943881,0.552096,0.941280,0.557626,0.936263,0.614421,0.012348,0.050141,0.995931,0.090940
2,[5/30/2017 12:56:00 PM],0.850207,0.604282,-0.00010,-0.00043,0.509777,0.389955,0.201990,0.999966,0.932317,...,0.927529,0.551058,0.924992,0.556860,0.920199,0.613874,0.013001,0.050166,0.995931,0.090935
3,[5/30/2017 12:56:00 PM],0.833468,0.603537,-0.00016,-0.00064,0.509437,0.389955,0.303992,0.999945,0.919029,...,0.914333,0.550166,0.911848,0.556202,0.907236,0.613404,0.013393,0.050190,0.995931,0.090946
4,[5/30/2017 12:56:00 PM],0.821151,0.602997,-0.00021,-0.00085,0.509210,0.389955,0.400994,0.999928,0.908082,...,0.903461,0.549414,0.901018,0.555648,0.896556,0.613008,0.013681,0.050208,0.995931,0.090954
5,[5/30/2017 12:56:00 PM],0.812305,0.602560,-0.00026,-0.00105,0.508985,0.389955,0.501999,0.999910,0.899003,...,0.894445,0.548767,0.892037,0.555171,0.887698,0.612667,0.013918,0.050225,0.996629,0.090962


# Save file

In [25]:
features = ['Voltage', 'Current', 'Power', 'Battery_Temp_degC', 'Voltage_MA1000', 'Current_MA1000', 'Voltage_MA400', 'Current_MA400', 'Voltage_MA200', 'Current_MA200', 'Voltage_MA100', 'Current_MA100', 'Voltage_MA50', 'Current_MA50', 'Voltage_MA10', 'Current_MA10', 'Voltage_grad', 'Current_grad', 'Battery_Temp_grad', 'Power_grad']
target = ['SoC']

In [26]:
df = df_drive_cycle[features + target]

In [28]:
df.to_csv("../data/data_panasonic_0.0.2.csv")

In [31]:
json.dump(norm_basis, open("../data/norm_basis_0.0.2.json", "w"))